<a href="https://colab.research.google.com/github/pamelladon/FTW3-Day3/blob/master/NYC_House_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Link to Dataset Assignments](https://docs.google.com/spreadsheets/d/10OKNCGU9dFmqgVDyWDduN_dHG7RPhSqJJt3fMGUaQwI/edit?usp=sharing)

# New Section

[Data Descriptions](https://colab.research.google.com/drive/14i0RSqKRLowamtV5DjUxqlbf7QmU5eRm)

In [0]:
# IMPORT GENERIC PACKAGES
import numpy as np # numerical calc package
import pandas as pd # holds data
import matplotlib.pyplot as plt # plotting library
import seaborn as sns # pretty plotting

# plotting config
sns.set(style='white', rc={'figure.figsize':(20,10)})

from sklearn.linear_model import LinearRegression # linear regression package
from sklearn.model_selection import train_test_split # split dataset
from sklearn.metrics import mean_squared_error as mse # Measurement metric

## Objective:  Predict Sales Using Advertising Costs From Different Digital Platforms

### Read Data

In [0]:
# read data into a DataFrame
data = pd.read_csv('nyc-rolling-sales (1).csv', index_col=0)

# Look at the Data

In [3]:
data.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,10009,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,,10009,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,,10009,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,10009,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,10009,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [4]:
!pip install pandas_profiling

In [0]:
import pandas_profiling as pp
#assign pandas profiling as pp to show data profiles

In [6]:
report = pp.ProfileReport(data)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


In [0]:
report.to_file('data_assessment.html')

In [8]:
# Get number of (rows, columns)
data.shape

(84548, 21)

In [9]:
# Get first 5 rows
data.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,,10009,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,,10009,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,,10009,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,,10009,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,,10009,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


CLEANING DATA

In [10]:
data['SALE PRICE'].head()

4    6625000
5        -  
6        -  
7    3936272
8    8000000
Name: SALE PRICE, dtype: object

In [0]:
#change data to number
data['SALE PRICE']=pd.to_numeric(data['SALE PRICE'], errors='coerce')

In [12]:
data['SALE PRICE'].head()

4    6625000.0
5          NaN
6          NaN
7    3936272.0
8    8000000.0
Name: SALE PRICE, dtype: float64

In [13]:
#identifies if data is na or not
data['SALE PRICE'].isna()

4       False
5        True
6        True
7       False
8       False
        ...  
8409    False
8410    False
8411    False
8412    False
8413    False
Name: SALE PRICE, Length: 84548, dtype: bool

In [14]:
data['SALE PRICE'].isna().sum() 
#sum all values that does not have a value

14561

In [0]:
data['SALE PRICE'] = data['SALE PRICE'].fillna(0) #assigns blank with 0

In [16]:
data['SALE PRICE'].isna().sum() 

0

In [17]:
sns.distplot(data['SALE PRICE'])

In [0]:
#remove rows in data where sales price is zero (because too much zero does not give good data) rename to newdata
newdata = data['SALE PRICE'] > 0

In [19]:
data['GROSS SQUARE FEET'].describe()

count     84548
unique     5691
top         -  
freq      27612
Name: GROSS SQUARE FEET, dtype: object

In [20]:
data['GROSS SQUARE FEET'].head

<bound method NDFrame.head of 4        6440
5       18690
6        7803
7        6794
8        4615
        ...  
8409     2575
8410     2377
8411     1496
8412    64117
8413     2400
Name: GROSS SQUARE FEET, Length: 84548, dtype: object>

In [0]:
#change data to number
data['GROSS SQUARE FEET']=pd.to_numeric(data['GROSS SQUARE FEET'], errors='coerce')

In [22]:
data['GROSS SQUARE FEET'].isna()

4       False
5       False
6       False
7       False
8       False
        ...  
8409    False
8410    False
8411    False
8412    False
8413    False
Name: GROSS SQUARE FEET, Length: 84548, dtype: bool

In [23]:
data['GROSS SQUARE FEET'].isna().sum()

27612

In [0]:
data['GROSS SQUARE FEET'] = data['GROSS SQUARE FEET'].fillna(0)

In [25]:
data['GROSS SQUARE FEET'].describe

<bound method NDFrame.describe of 4        6440.0
5       18690.0
6        7803.0
7        6794.0
8        4615.0
         ...   
8409     2575.0
8410     2377.0
8411     1496.0
8412    64117.0
8413     2400.0
Name: GROSS SQUARE FEET, Length: 84548, dtype: float64>

In [26]:
data['GROSS SQUARE FEET'].isna().sum()

0

In [27]:
data.describe

<bound method NDFrame.describe of       BOROUGH   NEIGHBORHOOD  ...  SALE PRICE            SALE DATE
4           1  ALPHABET CITY  ...   6625000.0  2017-07-19 00:00:00
5           1  ALPHABET CITY  ...         0.0  2016-12-14 00:00:00
6           1  ALPHABET CITY  ...         0.0  2016-12-09 00:00:00
7           1  ALPHABET CITY  ...   3936272.0  2016-09-23 00:00:00
8           1  ALPHABET CITY  ...   8000000.0  2016-11-17 00:00:00
...       ...            ...  ...         ...                  ...
8409        5        WOODROW  ...    450000.0  2016-11-28 00:00:00
8410        5        WOODROW  ...    550000.0  2017-04-21 00:00:00
8411        5        WOODROW  ...    460000.0  2017-07-05 00:00:00
8412        5        WOODROW  ...  11693337.0  2016-12-21 00:00:00
8413        5        WOODROW  ...     69300.0  2016-10-27 00:00:00

[84548 rows x 21 columns]>

In [28]:
data.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE'],
      dtype='object')

In [29]:
data.columns.dtype

dtype('O')

In [30]:
data.dtypes

BOROUGH                             int64
NEIGHBORHOOD                       object
BUILDING CLASS CATEGORY            object
TAX CLASS AT PRESENT               object
BLOCK                               int64
LOT                                 int64
EASE-MENT                          object
BUILDING CLASS AT PRESENT          object
ADDRESS                            object
APARTMENT NUMBER                   object
ZIP CODE                            int64
RESIDENTIAL UNITS                   int64
COMMERCIAL UNITS                    int64
TOTAL UNITS                         int64
LAND SQUARE FEET                   object
GROSS SQUARE FEET                 float64
YEAR BUILT                          int64
TAX CLASS AT TIME OF SALE           int64
BUILDING CLASS AT TIME OF SALE     object
SALE PRICE                        float64
SALE DATE                          object
dtype: object

In [31]:
data['LAND SQUARE FEET']=pd.to_numeric(data['LAND SQUARE FEET'], errors='coerce')
data['LAND SQUARE FEET'] = data['LAND SQUARE FEET'].fillna(0)
data['LAND SQUARE FEET'].isna().sum()

data['SALE DATE']=pd.to_datetime(data['SALE DATE'], errors='coerce')

data.dtypes

BOROUGH                                    int64
NEIGHBORHOOD                              object
BUILDING CLASS CATEGORY                   object
TAX CLASS AT PRESENT                      object
BLOCK                                      int64
LOT                                        int64
EASE-MENT                                 object
BUILDING CLASS AT PRESENT                 object
ADDRESS                                   object
APARTMENT NUMBER                          object
ZIP CODE                                   int64
RESIDENTIAL UNITS                          int64
COMMERCIAL UNITS                           int64
TOTAL UNITS                                int64
LAND SQUARE FEET                         float64
GROSS SQUARE FEET                        float64
YEAR BUILT                                 int64
TAX CLASS AT TIME OF SALE                  int64
BUILDING CLASS AT TIME OF SALE            object
SALE PRICE                               float64
SALE DATE           

In [0]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


# integer encode
le = LabelEncoder()
data['NEIGHBORHOOD'] = le.fit_transform(data['NEIGHBORHOOD'])
data['BUILDING CLASS CATEGORY'] = le.fit_transform(data['BUILDING CLASS CATEGORY'])
data['TAX CLASS AT PRESENT'] = le.fit_transform(data['TAX CLASS AT PRESENT'])
data['RESIDENTIAL UNITS'] = le.fit_transform(data['RESIDENTIAL UNITS'])
data['COMMERCIAL UNITS'] = le.fit_transform(data['COMMERCIAL UNITS'])
data['LAND SQUARE FEET'] = le.fit_transform(data['LAND SQUARE FEET'])
data['GROSS SQUARE FEET'] = le.fit_transform(data['GROSS SQUARE FEET'])
data['TAX CLASS AT TIME OF SALE'] = le.fit_transform(data['TAX CLASS AT TIME OF SALE'])
data['BUILDING CLASS AT TIME OF SALE'] = le.fit_transform(data['BUILDING CLASS AT TIME OF SALE'])



In [34]:
df = ['BUILDING CLASS AT PRESENT','NEIGHBORHOOD','BUILDING CLASS CATEGORY', 'TAX CLASS AT PRESENT', 'RESIDENTIAL UNITS','COMMERCIAL UNITS','LAND SQUARE FEET', 'GROSS SQUARE FEET', 'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE']
pd.get_dummies(df)

,BUILDING CLASS AT PRESENT,BUILDING CLASS AT TIME OF SALE,BUILDING CLASS CATEGORY,COMMERCIAL UNITS,GROSS SQUARE FEET,LAND SQUARE FEET,NEIGHBORHOOD,RESIDENTIAL UNITS,TAX CLASS AT PRESENT,TAX CLASS AT TIME OF SALE
0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,1,0,0
5,0,0,0,1,0,0,0,0,0,0
6,0,0,0,0,0,1,0,0,0,0
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,1
9,0,1,0,0,0,0,0,0,0,0


In [35]:
data.head()

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
4,1,1,6,6,392,6,,C2,153 AVENUE B,,10009,5,0,5,798,3857,1900,1,15,6625000.0,2017-07-19
5,1,1,6,5,399,26,,C7,234 EAST 4TH STREET,,10009,28,3,31,3329,4801,1900,1,20,0.0,2016-12-14
6,1,1,6,5,399,39,,C7,197 EAST 3RD STREET,,10009,16,1,17,1366,4077,1900,1,20,0.0,2016-12-09
7,1,1,6,7,402,21,,C4,154 EAST 7TH STREET,,10009,10,0,10,1424,3930,1913,1,17,3936272.0,2016-09-23
8,1,1,6,6,404,55,,C2,301 EAST 10TH STREET,,10009,6,0,6,1520,3310,1900,1,15,8000000.0,2016-11-17


In [69]:
he = OneHotEncoder()
data['NEIGHBORHOOD'] = he.fit_transform(data['NEIGHBORHOOD']).toarray()
data['BUILDING CLASS CATEGORY'] = he.fit_transform(data['BUILDING CLASS CATEGORY']).toarray()
data['TAX CLASS AT PRESENT'] = he.fit_transform(data['TAX CLASS AT PRESENT']).toarray()
data['RESIDENTIAL UNITS'] = he.fit_transform(data['RESIDENTIAL UNITS']).toarray()
data['COMMERCIAL UNITS'] = he.fit_transform(data['COMMERCIAL UNITS']).toarray()
data['LAND SQUARE FEET'] = he.fit_transform(data['LAND SQUARE FEET']).toarray()
data['GROSS SQUARE FEET'] = he.fit_transform(data['GROSS SQUARE FEET']).toarray()
data['TAX CLASS AT TIME OF SALE'] = he.fit_transform(data['TAX CLASS AT TIME OF SALE']).toarray()
data['BUILDING CLASS AT TIME OF SALE'] = he.fit_transform(data['BUILDING CLASS AT TIME OF SALE']).toarray()

ValueError: ignored

In [37]:
data.dtypes

BOROUGH                                    int64
NEIGHBORHOOD                               int64
BUILDING CLASS CATEGORY                    int64
TAX CLASS AT PRESENT                       int64
BLOCK                                      int64
LOT                                        int64
EASE-MENT                                 object
BUILDING CLASS AT PRESENT                 object
ADDRESS                                   object
APARTMENT NUMBER                          object
ZIP CODE                                   int64
RESIDENTIAL UNITS                          int64
COMMERCIAL UNITS                           int64
TOTAL UNITS                                int64
LAND SQUARE FEET                           int64
GROSS SQUARE FEET                          int64
YEAR BUILT                                 int64
TAX CLASS AT TIME OF SALE                  int64
BUILDING CLASS AT TIME OF SALE             int64
SALE PRICE                               float64
SALE DATE           

In [0]:
data['BUILDING CLASS AT PRESENT'] = le.fit_transform(data['BUILDING CLASS AT PRESENT'])

In [39]:
data.dtypes

BOROUGH                                    int64
NEIGHBORHOOD                               int64
BUILDING CLASS CATEGORY                    int64
TAX CLASS AT PRESENT                       int64
BLOCK                                      int64
LOT                                        int64
EASE-MENT                                 object
BUILDING CLASS AT PRESENT                  int64
ADDRESS                                   object
APARTMENT NUMBER                          object
ZIP CODE                                   int64
RESIDENTIAL UNITS                          int64
COMMERCIAL UNITS                           int64
TOTAL UNITS                                int64
LAND SQUARE FEET                           int64
GROSS SQUARE FEET                          int64
YEAR BUILT                                 int64
TAX CLASS AT TIME OF SALE                  int64
BUILDING CLASS AT TIME OF SALE             int64
SALE PRICE                               float64
SALE DATE           

In [0]:
data.head()

# How are the features correlated to each other?

In [41]:
correlation_coefficients = data.corr()
correlation_coefficients

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE
BOROUGH,1.000000,-0.144981,-0.317936,-0.361609,0.417001,-0.277848,-0.303971,0.256564,-0.035805,-0.036903,-0.023346,0.386896,0.114237,0.108303,-0.315163,-0.311548,-0.070856
NEIGHBORHOOD,-0.144981,1.000000,0.034504,0.040401,-0.015612,0.064599,0.022864,-0.115576,-0.018969,-0.012383,-0.009389,-0.033979,-0.104636,-0.034196,0.036727,0.023519,0.004019
BUILDING CLASS CATEGORY,-0.317936,0.034504,1.000000,0.863094,-0.292885,0.450135,0.644088,-0.163074,-0.064613,0.118785,-0.001565,-0.314957,-0.213852,-0.258683,0.935198,0.666040,0.049954
TAX CLASS AT PRESENT,-0.361609,0.040401,0.863094,1.000000,-0.336206,0.319449,0.586269,-0.079207,0.001099,0.184747,0.030853,-0.298261,-0.119895,-0.152360,0.932756,0.556027,0.068597
BLOCK,0.417001,-0.015612,-0.292885,-0.336206,1.000000,-0.229118,-0.296627,0.234063,-0.028936,-0.032303,-0.015888,0.307723,0.066702,0.071262,-0.287300,-0.300279,-0.054573
LOT,-0.277848,0.064599,0.450135,0.319449,-0.229118,1.000000,0.657077,-0.145802,-0.068209,-0.055031,-0.033169,-0.401427,-0.359712,-0.289463,0.296680,0.681074,0.010795
BUILDING CLASS AT PRESENT,-0.303971,0.022864,0.644088,0.586269,-0.296627,0.657077,1.000000,-0.148283,-0.072735,0.085434,-0.018224,-0.386512,-0.303297,-0.359724,0.510355,0.973161,0.038074
ZIP CODE,0.256564,-0.115576,-0.163074,-0.079207,0.234063,-0.145802,-0.148283,1.000000,0.019467,0.000008,0.005415,0.054126,0.132627,0.330708,-0.142915,-0.243183,-0.029644
RESIDENTIAL UNITS,-0.035805,-0.018969,-0.064613,0.001099,-0.028936,-0.068209,-0.072735,0.019467,1.000000,0.090943,0.662830,0.271753,0.420519,0.052063,-0.020532,-0.076823,0.097147
COMMERCIAL UNITS,-0.036903,-0.012383,0.118785,0.184747,-0.032303,-0.055031,0.085434,0.000008,0.090943,1.000000,0.329750,0.175752,0.279093,0.030424,0.215620,0.083737,0.202213


In [42]:
# Create heatmap of correlation coefficients
sns.heatmap(correlation_coefficients, annot=True, cmap='Reds')

# Visualize Distributions and Relationships

In [43]:
sns.pairplot(data, height=5)

# Build a Machine Learning Model

# Linear Regression  
Linear regression is the simplest method of predicting continuous variables. Its all about establishing a relationship between dependent variable (*Y*) and one or more independent variables (*X*) using a best fit straight line (also known as regression line).

Mathematically,

$\begin {equation}
h(x_i) = y = m_i x_i + b
\end {equation}$

where $m_i$ is the linear coefficient (slope) and $b$ is the intercept.

### Why learn linear regression?

- Easy to use,
- Used everywhere,
- Runs quickly,
- interpretable,
- most basic form of machine learning.  




### For now, we use all three variables to predict sales  


In [44]:
data.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY',
       'TAX CLASS AT PRESENT', 'BLOCK', 'LOT', 'EASE-MENT',
       'BUILDING CLASS AT PRESENT', 'ADDRESS', 'APARTMENT NUMBER', 'ZIP CODE',
       'RESIDENTIAL UNITS', 'COMMERCIAL UNITS', 'TOTAL UNITS',
       'LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT',
       'TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE',
       'SALE PRICE', 'SALE DATE'],
      dtype='object')

In [0]:
# Define Features
features = ['NEIGHBORHOOD','GROSS SQUARE FEET','TAX CLASS AT TIME OF SALE', 'BUILDING CLASS AT TIME OF SALE','TAX CLASS AT PRESENT', 'BUILDING CLASS AT PRESENT']

# Set X
X = data[features]

In [0]:
# Set y
y = data['SALE PRICE']

In [0]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 42)

In [0]:
# Initialize model
model = LinearRegression() 


In [254]:
# Fit Model
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
# get coefficients
coefficient = model.coef_

# get intercept
intercept = model.intercept_

In [256]:
# print coefficient
coefficient

array([  1804.16474995,   1130.67340591, 934294.11247813,  11675.98942105,
        -7103.03777853,  -2523.96581953])

In [257]:
# print intercept
intercept

-1149505.7380780585

In [258]:
# calculate the R-squared
model.score(X_test, y_test)

0.03541445848649738

In [0]:
y_predicted = model.predict(X_test)

### Model Error

In [260]:
np.sqrt(mse(y_predicted, y_test)) # Root mean squared error

5154715.084013985